In [1]:
import pandas as pd

ips = ["18.219.5.43", "18.218.115.60", "18.219.211.138", "172.31.69.15", "18.218.229.235", "172.31.69.17", "172.31.69.9", "172.31.0.2", "172.31.69.25", "18.217.165.70", "18.218.55.126", "18.221.219.4", "13.59.126.31", "172.31.69.8", "18.219.9.1", "172.31.69.28", "13.58.98.64", "18.219.193.20", "18.216.24.42", "18.219.32.43", "18.218.11.51", "52.14.136.135", "172.31.69.14", "172.31.69.13", "18.216.200.189"]

In [2]:
from dotenv import load_dotenv
from os import getenv
from datasets import load_dataset
import pandas as pd

load_dotenv()
HUGGING_FACE_READ_TOKEN = getenv("HUGGING_FACE_READ_TOKEN")

index = 0
dataset = load_dataset("Jetlime/NF-CSE-CIC-IDS2018-v2", streaming=False, split="test")
dataset = dataset.to_pandas()

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

In [3]:
row_index = dataset.loc[dataset['input'] == "IPV4_SRC_ADDR: 18.219.9.1, L4_SRC_PORT: 52156, IPV4_DST_ADDR: 172.31.69.28, L4_DST_PORT: 80, PROTOCOL: 6, L7_PROTO: 7.0, IN_BYTES: 478, IN_PKTS: 5, OUT_BYTES: 1147, OUT_PKTS: 5, TCP_FLAGS: 219, CLIENT_TCP_FLAGS: 219, SERVER_TCP_FLAGS: 27, FLOW_DURATION_MILLISECONDS: 4294940, DURATION_IN: 26, DURATION_OUT: 26, MIN_TTL: 127, MAX_TTL: 127, LONGEST_FLOW_PKT: 975, SHORTEST_FLOW_PKT: 40, MIN_IP_PKT_LEN: 40, MAX_IP_PKT_LEN: 975, SRC_TO_DST_SECOND_BYTES: 478.0, DST_TO_SRC_SECOND_BYTES: 1147.0, RETRANSMITTED_IN_BYTES: 0, RETRANSMITTED_IN_PKTS: 0, RETRANSMITTED_OUT_BYTES: 0, RETRANSMITTED_OUT_PKTS: 0, SRC_TO_DST_AVG_THROUGHPUT: 136000, DST_TO_SRC_AVG_THROUGHPUT: 336000, NUM_PKTS_UP_TO_128_BYTES: 8, NUM_PKTS_128_TO_256_BYTES: 0, NUM_PKTS_256_TO_512_BYTES: 1, NUM_PKTS_512_TO_1024_BYTES: 1, NUM_PKTS_1024_TO_1514_BYTES: 0, TCP_WIN_MAX_IN: 65535, TCP_WIN_MAX_OUT: 26883, ICMP_TYPE: 0, ICMP_IPV4_TYPE: 0, DNS_QUERY_ID: 0, DNS_QUERY_TYPE: 0, DNS_TTL_ANSWER: 0, FTP_COMMAND_RET_CODE: 0"].index[0]
filtered_rows = dataset[(dataset['input'].str.contains('18.219.9.1')) | (dataset['input'].str.contains('172.31.69.28'))]
preceding_rows = filtered_rows[filtered_rows.index < row_index].tail(10)

In [4]:
import requests

data = {}

ip_intel = ["IP flagged for SSH brute-force attempts",
"Botnet communication detected",
"Phishing email source",
"Part of a DDoS attack",
"Recognized as part of a DDoS botnet",
"SSH brute-force activity observed",
"Source of spam email activity",
"IP flagged in multiple attack patterns",
"Involved in phishing campaigns",
"Malicious activity linked to malware distribution",
"Source of repeated brute-force attempts",
"Flagged for suspicious DNS requests",
"Involved in botnet communication",
"Known for SSH brute-force attacks",
"Source of spam emails",
"Recognized for involvement in DDoS attacks",
]

def get_location(ip_address):
    response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()
    location_data = {
        "ip": ip_address,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name")
    }
    return location_data

for ip in ips:
    data[ip] = {}
    location = get_location(ip)
    data[ip]['location'] = f"{location['city']}, {location['region']}, {location['country']}"
    print(location['country'])
    if location['country'] == None:
        data[ip]['intelligence'] = ''
    else:
        data[ip]['intelligence'] = ip_intel.pop()
data

United States
United States
United States
None
United States
None
None
None
None
United States
United States
United States
United States
None
United States
None
United States
United States
United States
United States
United States
United States
None
None
United States


{'18.219.5.43': {'location': 'Hilliard, Ohio, United States',
  'intelligence': 'Recognized for involvement in DDoS attacks'},
 '18.218.115.60': {'location': 'Hilliard, Ohio, United States',
  'intelligence': 'Source of spam emails'},
 '18.219.211.138': {'location': 'Hilliard, Ohio, United States',
  'intelligence': 'Known for SSH brute-force attacks'},
 '172.31.69.15': {'location': 'None, None, None', 'intelligence': ''},
 '18.218.229.235': {'location': 'Hilliard, Ohio, United States',
  'intelligence': 'Involved in botnet communication'},
 '172.31.69.17': {'location': 'None, None, None', 'intelligence': ''},
 '172.31.69.9': {'location': 'None, None, None', 'intelligence': ''},
 '172.31.0.2': {'location': 'None, None, None', 'intelligence': ''},
 '172.31.69.25': {'location': 'None, None, None', 'intelligence': ''},
 '18.217.165.70': {'location': 'Hilliard, Ohio, United States',
  'intelligence': 'Flagged for suspicious DNS requests'},
 '18.218.55.126': {'location': 'Hilliard, Ohio, Un

In [5]:
# Convert the dictionary to a DataFrame
ip_df = pd.DataFrame.from_dict(data, orient='index')

# Save the DataFrame to a CSV file
csv_path = 'ip_information.csv'
ip_df.to_csv(csv_path)

csv_path

'ip_information.csv'